## Day 1

In [7]:
depths = [int(d.strip("\n")) for d in open('day1.input', 'r').readlines()]

In [9]:
# number of depth values which are larger than the previous
print(sum((int(v) > depths[i-1] for i, v in enumerate(depths) if i > 0)))

1692


In [14]:
windowed = [sum(depths[i:i+3]) for i in range(len(depths)-2)]

In [19]:
print(sum((int(v) > windowed[i-1] for i, v in enumerate(windowed) if i > 0)))

1724


## Day 2

In [2]:
navs = [d.strip("\n").split() for d in open('day2.input', 'r').readlines()]

In [7]:
forward = sum((int(n[1]) for n in navs if n[0] == "forward"))
up = sum((int(n[1]) for n in navs if n[0] == "up"))
down = sum((int(n[1]) for n in navs if n[0] == "down"))
print((down - up)*forward)

1636725


In [14]:
horizontal = 0
aim = 0
depth = 0
for nav in navs:
    if nav[0] == "down":
        aim += int(nav[1])
    elif nav[0] == "up":
        aim -= int(nav[1])
    elif nav[0] == "forward":
        horizontal += int(nav[1])
        depth += int(nav[1]) * aim
    else:
        print(f"ohoh... {nav}")

In [15]:
print(horizontal * depth)

1872757425


## Day 3

In [42]:
report = [b.strip("\n") for b in open('day3.input').readlines()]
demo = ["00100","11110","10110","10111","10101","01111","00111","11100","10000","11001","00010","01010"]

In [37]:
def get_counts(report: list) -> dict:
    counts = {i: {'0': 0, '1': 0} for i in range(len(report[0]))}
    for binary in report:
        for i, d in enumerate(binary):
            counts[i][d] += 1
    return counts


def solve_day_3(report: list) -> int:    
    counts = get_counts(report)
    gamma = ''
    epsilon = ''
    for c in counts.values():
        s = sorted(c.items(), key=lambda x: x[1])
        epsilon = epsilon + s[0][0]
        gamma = gamma + s[1][0]
    return int(gamma, 2) * int(epsilon, 2)

In [43]:
assert solve_day_3(demo) == 198

In [39]:
solve_day_3(report) # 4191876

4191876

In [97]:
def oxygen(report: list) -> int:
    counts = get_counts(report)
    oxygen = ''
    for i in range(len(report[0])):
        if counts[i]['0'] == counts[i]['1']:
            oxygen += '1'
        else:
            s = sorted(counts[i].items(), key=lambda x: x[1])
            oxygen  += s[1][0]
        report = [r for r in report if r.startswith(oxygen)]
        if len(report) == 1:
            return int(oxygen, 2)
        counts = get_counts(report)

def co2(report: list) -> int:
    counts = get_counts(report)
    co2 = ''
    for i in range(len(report[0])):
        if counts[i]['0'] == counts[i]['1']:
            co2 += '0'
        else:
            s = sorted(counts[i].items(), key=lambda x: x[1])
            co2  += s[0][0]
        report = [r for r in report if r.startswith(co2)]
        if len(report) == 1:
            return int(report[0], 2)
        counts = get_counts(report)


In [98]:
assert oxygen(demo) == 23
assert co2(demo) == 10

In [99]:
print(oxygen(report) * co2(report)) # 3414905

3414905


## Day 4

In [1]:
import numpy as np

In [91]:
def read_input(file: str) -> tuple:
    with open(file, 'r') as f:
        calls = [int(i) for i in f.readline().strip('\n').split(',')]
        puzzles = [p.strip().split('\n') for p in f.read().split("\n\n")]
    for i, p in enumerate(puzzles):
        puzzles[i] = np.array([r.strip().split() for r in p], dtype=int)
    marks = [np.zeros(p.shape) for p in puzzles]
    return calls, puzzles, marks

In [94]:
def check_mark(mark) -> bool:
    return len(np.where(mark.sum(axis=1) == 5)[0]) == 1 or len(np.where(mark.sum(axis=0) == 5)[0]) == 1

def solve_part_1(calls, puzzles, marks):
    for call in calls:
        for puzzle, mark in zip(puzzles, marks):
            mark[np.where(puzzle == call)] = 1
            if check_mark(mark):
                return puzzle[np.where(mark == 0)].sum() * call

In [96]:
assert solve_part_1(*read_input('day4.example.input')) == 4512

In [97]:
solve_part_1(*read_input('day4.input')) # 16674

16674

In [111]:
def solve_part_2(calls, puzzles, marks):
    solved = np.zeros([len(puzzles)])
    for call in calls:
        for puzzle, mark, index in zip(puzzles, marks, range(len(puzzles))):
            mark[np.where(puzzle == call)] = 1
            if check_mark(mark):
                solved[index] = 1
                if sum(solved) == len(puzzles):
                    return  puzzle[np.where(mark == 0)].sum() * call

In [112]:
assert solve_part_2(*read_input('day4.example.input')) == 1924

In [113]:
solve_part_2(*read_input('day4.input')) # 7075

7075

## Day 5

In [1]:
import numpy as np

In [73]:
def read_input(file: str):
    with open(file, 'r') as f:
        vents = [line.strip("\n").split(" -> ") for line in f.readlines()]
    for i, p in enumerate(vents):
        vents[i] =[int(i) for c in p for i in c.split(',')]
    
    maxes = np.array(vents).max(axis=0)
    mask = np.zeros([max(maxes[1], maxes[3])+1, max(maxes[0], maxes[2])+1])
    return vents, mask

In [123]:
def solve(vents, mask, incl_diagonal = False) -> int:
    for x1, y1, x2, y2 in vents:
        xmin, xmax = min(x1,x2), max(x1,x2)
        ymin, ymax = min(y1,y2), max(y1,y2)
        if x1 == x2:
            mask[ymin:ymax+1, xmin] += 1
        elif y1 == y2:
            mask[ymin, xmin:xmax+1] += 1
        elif incl_diagonal:
            xrange = range(x1, x2+1) if x2 > x1 else range(x1, x2-1, -1)
            yrange = range(y1, y2+1) if y2 > y1 else range(y1, y2-1, -1)
            for x, y in zip(xrange, yrange):
                mask[y,x] += 1
    return sum(sum(mask > 1))        

In [124]:
assert solve(*read_input('day5.example.input')) == 5

In [127]:
solve(*read_input('day5.input')) # 6572

6572

In [126]:
assert solve(*read_input('day5.example.input'), True) == 12

In [128]:
solve(*read_input('day5.input'), True) # 21466

21466

## Day 6

In [1]:
import numpy as np

In [2]:
def read_input(file: str):
    with open(file, 'r') as f:
        data = f.readline()
    return [int(d) for d in data.strip("\n").split(",")]

In [247]:
def solve(fishes, days = 80):
    length = 0
    for i in range(len(fishes)):
        fish = np.array([fishes[i]])
        for day in range(days):
            fish = fish -1
            reset = np.where(fish == -1)
            fish[reset] = 6
            fish = np.append(fish, np.ones(len(reset[0]))*8)
        length += len(fish)
    return length

In [248]:
assert solve(np.array([3,4,3,1,2])) == 5934

In [249]:
solve(np.array(read_input('day6.input'))) # 390923

390923

In [243]:
def solve2(fishes, days = 80):
    stages = [0] * 9
    for fish in fishes:
        stages[fish] += 1

    for day in range(days):
        today = day % len(stages)
        stages[(today + 7) % len(stages)] += stages[today]
    
    return sum(stages)

In [245]:
assert solve2([3,4,3,1,2], 256) == 26984457539

In [246]:
solve2(read_input('day6.input'), 256) # 1749945484935

1749945484935

In [ ]:
def solve2(fishes, days = 80):
    stages = [0] * 9
    for fish in fishes:
        stages[fish] += 1

    for day in range(days):
        today = day % len(stages)
        stages[(today + 7) % len(stages)] += stages[today]
    
    return sum(stages)

## Day 7

In [250]:
import numpy as np

In [260]:
def read_input(file: str):
    with open(file, 'r') as f:
        data = f.readline()
    return [int(d) for d in data.strip("\n").split(",")]

In [302]:
def solve_part1(positions: list) -> int:
    optimum = int(np.median(positions))
    return sum(np.abs(np.array(positions) - optimum))

In [303]:
assert solve_part1([16,1,2,0,4,2,7,1,2,14]) == 37

In [304]:
solve_part1(read_input('day7.input')) # 335330

335330

In [330]:
def solve_part2(positions: list) -> int:
    min_fuel = 10000000000000
    for i in range(min(positions), max(positions)):
        if (fuel := sum([sum(range(abs(p-i)+1)) for p in positions])) > min_fuel:
            return min_fuel
        else:
            min_fuel = fuel

In [331]:
assert solve_part2([16,1,2,0,4,2,7,1,2,14]) == 168

In [332]:
solve_part2(read_input('day7.input')) # 92439766

92439766

## Day 8

In [1]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=8)

In [5]:
data = [p.strip().split(" | ") for p in puzzle.input_data.split('\n')]

In [13]:
def read_input(file: str):
    with open(file, 'r') as f:
        patterns = f.readlines()
    return [pattern.strip().split(" | ") for pattern in patterns]

In [7]:
def solve_part1(patterns: list) -> int:
    unique = 0
    digits = []
    _ = [digits.extend(d[1].split(" ")) for d in patterns]
    for p in patterns:
        unique += sum((len(d) in [2,3,4,7] for d in p[1].split(" ")))
    return unique

In [14]:
assert solve_part1(read_input('day8.example.input')) == 26

In [12]:
puzzle.answer_a = solve_part1(data) # 387

In [18]:
def create_pattern_map(pattern: str) -> dict:
    patterns = pattern.split()
    pattern_map = {}
    for p in patterns:
        if len(p) == 2:
            pattern_map[1] = p
        elif len(p) == 4:
            pattern_map[4] = p
        elif len(p) == 3:
            pattern_map[7] = p
        elif len(p) == 7:
            pattern_map[8] = p
    m = {}
    for s in pattern_map[1]:
        m[s] = [p for p in patterns if s not in p]
    for k, v in m.items():
        if len(v) == 1:
            pattern_map[2] = v[0]
        else:
            for iv in v:
                pattern_map[len(iv)] = iv 
    zeronine = []
    for p in patterns:
        if len(p) == 5 and p not in pattern_map.values():
            pattern_map[3] = p
        if len(p) == 6 and p not in pattern_map.values():
            zeronine.append(p)
    for k in 'abcdefg':
        if k not in zeronine[0]:
            if k in pattern_map[3]:
                pattern_map[9] = zeronine[1]
                pattern_map[0] = zeronine[0]
            else:
                pattern_map[9] = zeronine[0]
                pattern_map[0] = zeronine[1]
            break
            
    return {''.join(sorted(v)): k for k, v in pattern_map.items()}
    
def solve_part2(data: list) -> int:
    total = 0
    for pattern in data:
        mp = create_pattern_map(pattern[0])
        digits = [mp[''.join(sorted(d))] for d in pattern[1].split()]
        total += int(''.join([str(i) for i in digits]))
    return total

In [153]:
assert solve_part2(read_input('day8.example.input')) == 61229

In [19]:
solve_part2(data) # 986034

986034

## Day 9

In [1]:
import numpy as np
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=9)

In [101]:
data = puzzle.input_data.split()

In [7]:
example_data = [[int(d) for d in line.strip()] for line in open('day9.example.input').readlines()]

In [209]:
def find_lows(array: np.array) -> tuple:
    x, y = array.shape
    wr = np.ones((x+2, y+2))*9  # wrapper
    wr[1:x+1, 1:y+1] = array
    mask = np.zeros(wr.shape)
    for i in range(1, x+1):
        for j in range(1, y+1):
            xy = wr[i, j]
            mask[i, j] = all([xy < n for n in [wr[i-1, j], wr[i+1, j], wr[i, j-1], wr[i, j+1]]])            
    return wr, np.where(mask == 1)

def solve_part1(data: list):
    wrapper, lows = find_lows(np.array(data))
    return int(sum(wrapper[lows]+1))

In [210]:
assert solve_part1(example_data) == 15

In [211]:
solutiona = solve_part1([[int(d) for d in line.strip()] for line in data])

In [212]:
solutiona

607

In [168]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [273]:
def patch_size(wr: np.array, seeds: set):
    length = len(seeds)
    additions = []
    for x, y in seeds:
        additions.extend([(i, j) for i, j in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)] if wr[i, j] != 9])
    seeds.update(set(additions))
    if len(seeds) == length:
        return length
    return patch_size(wr, seeds)
                          
def solve_part2(data: list):
    wrapper, lows = find_lows(np.array(data))
    sizes = [patch_size(wrapper, {(x, y)}) for x, y in zip(*lows)]
    sizes = sorted(sizes, reverse=True)
    return np.product(sizes[:3])

In [274]:
assert solve_part2(example_data) == 1134

In [276]:
solutionb = solve_part2([[int(d) for d in line.strip()] for line in data])
solutionb

900864

In [277]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 9! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 10

In [29]:
import numpy as np
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=10)

In [10]:
example_data = [line.strip() for line in open('day10.example.input').readlines()]

In [31]:
charmap = {")": 3, "]": 57, "}": 1197, ">": 25137}
opensets = {"]": "[", "}": "{", ")": "(", ">": "<"}

def validate_line(line: str):
    
    series = []
    for c in line:
        if c in opensets.values():
            series.append(c)
        else:
            last_opening = series.pop()
            if opensets[c] == last_opening:
                continue
            else:
                return False, c
    return True, None

def solve_part1(data: list) -> int:
    syntax_error_score = 0
    for line in data:
        result, char = validate_line(line)
        if not result:
            syntax_error_score += charmap[char]
    return syntax_error_score

In [32]:
assert solve_part1(example_data) == 26397

In [25]:
solutiona = solve_part1(puzzle.input_data.split())
solutiona

344193

In [26]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [67]:
charmap = {")": 1, "]": 2, "}": 3, ">": 4}
closesets = {"[": "]", "{": "}", "(": ")", "<": ">"}

def calculate_completion(line: str):
    series = []
    for c in line:
        if c in charsets.keys():
            series.append(c)
        else:
            assert opensets[c] == series.pop()
    completion = [closesets[c] for c in series]
    completion.reverse()
    return "".join(completion)

def calculate_score(line: str) -> int:
    score = 0
    for c in line:
        score = score * 5 + charmap[c]
    return score

def solve_part2(data: list) -> int:
    completion_scores = []
    for line in data:
        result, char = validate_line(line)
        if result:
            completion_scores.append(calculate_score(calculate_completion(line)))
    completion_scores.sort()
    return completion_scores[int(np.floor(len(completion_scores)/2))]

In [68]:
assert solve_part2(example_data) == 288957

In [69]:
solutionb = solve_part2(puzzle.input_data.split())
solutionb

3241238967

In [70]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 10! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 11

In [269]:
import numpy as np
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=11)
data = np.array([[int(d) for d in line.strip()] for line in puzzle.input_data.split()])

In [77]:
import scipy.ndimage as ndimage

In [266]:
example_data = np.array([[int(d) for d in line.strip()] for line in open('day11.example.input').readlines()])

In [181]:
footprint = np.array([[1,1,1],
                      [1,0,1],
                      [1,1,1]])
def test_func(values):
    return len(np.where(values > 9)[0])

In [259]:
def take_step(data: np.array) -> tuple:
    flashes = 0
    data += 1
    flashed = len(np.where(data > 9)[0])
    while flashed != 0:
        mask = ndimage.generic_filter(data, test_func, footprint=footprint, mode='constant')
        data[np.where(data > 9)] = -1000
        data += mask
        flashed = len(np.where(data > 9)[0])
    flashed = np.where(data < 0)
    data[flashed] = 0
    return data, len(flashed[0])

def solve_part1(data: np.array, steps = 100) -> int:
    flashes = 0
    for step in range(steps):
        data, flash = take_step(data)
        flashes += flash
    return flashes

In [260]:
assert solve_part1(example_data) == 1656

In [261]:
solutiona = solve_part1(data)
solutiona

1785

In [262]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [264]:
def solve_part2(data: np.array) -> int:
    step = 0
    while len(np.where(data == 0)[0]) != 100:
        step += 1
        data, _ = take_step(data)
    return step

In [267]:
assert solve_part2(example_data) == 195

In [270]:
solutionb = solve_part2(data)
solutionb

354

In [271]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 11! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 12

In [275]:
import numpy as np
from aocd.models import Puzzle
from collections import defaultdict
puzzle = Puzzle(year=2021, day=12)

In [339]:
data = puzzle.input_data.split()

In [356]:
example_data = [line.strip() for line in open('day12.example3.input').readlines()]

In [350]:
def create_map(paths: list) -> dict:
    cave_map = defaultdict(list)
    for path in paths:
        a, b = path.split("-")
        cave_map[a].append(b)
        cave_map[b].append(a)
    return cave_map

def find_paths(cave_map: dict, paths: list) -> list:
    for path in paths.copy():
        if path[-1] == "end":
            continue
        paths.remove(path)
        options = cave_map[path[-1]]
        for option in options:
            if option.isupper():
                sub = path.copy()
                paths.append(sub + [option])
            elif option not in path and option != "start":
                sub = path.copy()
                paths.append(sub + [option])
    return paths

def solve_part1(data: list) -> int:
    mp = create_map(data)
    num_paths = 0
    paths = [["start"]]
    while len(paths) != num_paths:
        num_paths = len(paths)
        paths = find_paths(mp, paths)
    return len(paths)

In [357]:
assert solve_part1(example_data) == 226

In [352]:
solutiona = solve_part1(data)
solutiona

5228

In [353]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [364]:
from collections import Counter

def find_many_paths(cave_map: dict, paths: list) -> list:
    for path in paths.copy():
        if path[-1] == "end":
            continue
        paths.remove(path)
        options = cave_map[path[-1]]
        for option in options:
            if option == "start":
                continue
            elif option.isupper():
                sub = path.copy()
                paths.append(sub + [option])
            elif option not in path:
                sub = path.copy()
                paths.append(sub + [option])
            else:
                c = Counter(path)
                if any((k.islower() and v == 2 for k, v in c.items())):
                    continue
                sub = path.copy()
                paths.append(sub + [option])
    return paths

def solve_part2(data: list) -> int:
    mp = create_map(data)
    num_paths = 0
    paths = [["start"]]
    visited_twice = False
    while len(paths) != num_paths:
        num_paths = len(paths)
        paths = find_many_paths(mp, paths)
    return len(paths)

In [366]:
assert solve_part2([line.strip() for line in open('day12.example1.input').readlines()]) == 36

In [367]:
assert solve_part2([line.strip() for line in open('day12.example2.input').readlines()]) == 103

In [368]:
assert solve_part2([line.strip() for line in open('day12.example3.input').readlines()]) == 3509

In [369]:
solutionb = solve_part2(data)
solutionb

131228

In [370]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 12! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 13

In [58]:
import numpy as np
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=13)
data = puzzle.input_data.split("\n")

In [63]:
def interpret_data(data: list):
    xs = []
    ys = []
    folding_instructions = []
    for line in data:
        if line and line[0].isdigit():
            x, y = line.strip().split(",")
            xs.append(int(x))
            ys.append(int(y))
        elif line.startswith('fold'):
            instructions = line.strip("fold along ").strip().split("=")
            folding_instructions.append((instructions[0], int(instructions[1])))
    return xs, ys, folding_instructions

In [18]:
example_data = open('day13.example.input').readlines()

In [110]:
def solve(data: list, part1: bool = True):
    xs, ys, instructions = interpret_data(data)
    mp = np.zeros((max(xs)+1, max(ys)+1))
    mp[xs, ys] = 1
    for instruction in instructions:
        if instruction[0] == 'y':
            nwa = mp[:, :instruction[1]]
            nwb = np.fliplr(mp[:, instruction[1]+1:])
            if nwa.shape[1] == nwb.shape[1]:
                mp = nwa + nwb
            elif nwa.shape[1] > nwb.shape[1]:
                mp = nwa
                mp[:, -nwb.shape[1]:] += nwb
            elif nwa.shape[1] < nwb.shape[1]:
                mp = nwb
                mp[:, -nwa.shape[1]:] += nwa
        elif instruction[0] == 'x':
            nwa = mp[:instruction[1], :]
            nwb = np.flipud(mp[instruction[1]+1:, :])
            if nwa.shape[0] == nwb.shape[0]:
                mp = nwa + nwb
            elif nwa.shape[0] > nwb.shape[0]:
                mp = nwa
                mp[-nwb.shape[0]:, :] += nwb
            elif nwa.shape[0] < nwb.shape[0]:
                mp = nwb
                mp[-nwa.shape[0]:, :] += nwa
        if part1:
            return len(np.where(mp > 0)[0])
    return mp

In [112]:
assert solve(example_data) == 17

In [96]:
solutiona = solve(data)
solutiona

810

In [91]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [118]:
for row in np.transpose(solve(data, False)):
    string = ''
    for i in row:
        if i == 0:
            string += '.'
        else:
            string += '#'
    print(string)

#..#.#....###..#..#.###...##..####.###..
#..#.#....#..#.#..#.#..#.#..#.#....#..#.
####.#....###..#..#.###..#....###..#..#.
#..#.#....#..#.#..#.#..#.#.##.#....###..
#..#.#....#..#.#..#.#..#.#..#.#....#.#..
#..#.####.###...##..###...###.#....#..#.


In [120]:
solutionb = "HLBUBGFR"

In [121]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 13! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 14

In [76]:
from aocd.models import Puzzle
from collections import Counter, defaultdict
puzzle = Puzzle(year=2021, day=14)
data = puzzle.input_data.split("\n")

In [4]:
example_data = [line.strip() for line in open('day14.example.input').readlines()]

In [5]:
def interpret_data(data: list) -> tuple:
    template = data[0]
    rules = {}
    for line in data[2:]:
        rule = line.split(' -> ')
        rules[rule[0]] = rule[1]
    return template, rules

In [155]:
def step(template: str, rules: dict) -> str:
    blocks = []
    for i in range(len(template)-1):
        pair = template[i:i+2]
        blocks.append(pair[0] + rules[pair] + pair[1])
    template = blocks[0]
    return template + ''.join([b[1:] for b in blocks[1:]])
    
def solve_part1(data: list, steps: int = 10) -> int:
    template, rules = interpret_data(data)
    for s in range(steps):
        t = defaultdict(lambda: 0)
        template = step(template, rules)
    c = Counter(template).values()
    return max(c) - min(c)

def solve_part2(data: list, steps: int = 40) -> int:
    template, rules = interpret_data(data)
    occurence_rules = defaultdict(list)
    t = defaultdict(lambda: 0)

    for k, v in rules.items():
        occurence_rules[k[0]+v].append(k)
        occurence_rules[v+k[1]].append(k)

    for i in range(len(template)-1):
        t[template[i:i+2]] += 1

    for s in range(steps):
        o = defaultdict(lambda: 0)
        for k, v in occurence_rules.items():
            o[k] = sum([t[i] for i in v])
        t = o
    
    char_count = defaultdict(lambda: 0)
    for k, v in t.items():
        for c in k:
            char_count[c] += v
    char_count[template[0]] -= 1
    char_count[template[-1]] -= 1
    char_count = {k: int(v/2) for k, v in char_count.items()}
    char_count[template[0]] += 1
    char_count[template[-1]] += 1
 
    c = Counter(char_count).values()
    return max(c) - min(c)

In [156]:
assert solve_part1(example_data, 10) == 1588

In [13]:
solutiona = solve(data)
solutiona

2375

In [34]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [157]:
assert solve_part2(example_data) == 2188189693529

In [158]:
solutionb = solve_part2(data)
solutionb

1976896901756

In [159]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 14! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 15

In [1]:
from aocd.models import Puzzle
import numpy as np
puzzle = Puzzle(year=2021, day=15)
data = puzzle.input_data.split("\n")

In [2]:
example_data = [line.strip() for line in open('day15.example.input').readlines()]

In [3]:
def convert_to_array(data: list) -> np.array:
    data = [[int(d) for d in line] for line in data]
    return np.array(data)

def min_cost(cost):
    tc = np.zeros(cost.shape)
    x, y = cost.shape
    
    for i in range(1, x):
        tc[i][0] = tc[i-1][0] + cost[i][0]
  
    for j in range(1, y):
        tc[0][j] = tc[0][j-1] + cost[0][j]
  
    for i in range(1, y):
        for j in range(1, x):
            tc[i][j] = min(tc[i-1][j], tc[i][j-1]) + cost[i][j]
  
    for i in range(10):
        tc2 = tc.copy()

        for i in range(1, y):
            for j in range(1, x):
                tc2[i][j] = min(tc2[i-1][j], tc2[i][j-1], tc2[min(i+1, y-1)][j], tc2[i][min(j+1, x-1)]) + cost[i][j]

        if int(tc2[-1][-1]) >= int(tc[-1][-1]):
            return int(tc2[-1][-1])
    
        tc = tc2

def solve_part1(data):
    cost = convert_to_array(data)
    return min_cost(cost)

In [93]:
assert solve_part1(example_data) == 40

In [94]:
solutiona = solve_part1(data)
solutiona

811

In [32]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [4]:
def solve_part2(data):
    cost = convert_to_array(data)
    extended = cost
    extension = cost
    for i in range(4):
        extension = extension + 1
        extension[np.where(extension > 9)] = 1
        extended = np.concatenate((extended, extension), axis=1)
    extension = extended
    for i in range(4):
        extension = extension + 1
        extension[np.where(extension > 9)] = 1
        extended = np.concatenate((extended, extension), axis=0)
    return min_cost(extended)

In [5]:
assert solve_part2(example_data) == 315

In [6]:
solutionb = solve_part2(data)
solutionb

3012

In [91]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 15! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 16

In [87]:
from aocd.models import Puzzle
import numpy as np
puzzle = Puzzle(year=2021, day=16)
data = puzzle.input_data

In [12]:
def hex2bin(hexa: str) -> str:
    return ''.join([bin(int(h, 16))[2:].zfill(4) for h in hexa])

In [75]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

def read_literal_packet(signal) -> tuple:
    packet = {"version": int(signal[:3], 2), "type_id": int(signal[3:6], 2), "full_signal_length": 6}
    value = ''
    for group in chunks(signal[6:], 5):
        value += group[1:]
        packet["full_signal_length"] += 5
        if group [0] == "0":
            packet["literal"] = int(value, 2)
            return signal[packet["full_signal_length"]:], packet

def read_operator_packet(binary):
    packet = {"version": int(binary[:3], 2), "type_id": int(binary[3:6], 2), "length_type_id": binary[6], "subpackets": []}
    if packet["length_type_id"] == "0":
        packet_bit_length = int(binary[7:22], 2)
        subpackets = binary[22:22+packet_bit_length]
        while len(subpackets) != 0:
            subpackets, p = decode_hex(subpackets)
            packet["subpackets"].append(p)
        return binary[22+packet_bit_length: ], packet
    elif packet["length_type_id"] == "1":
        packet_numbers = int(binary[7:18], 2)
        binary = binary[18:]
        while len(packet["subpackets"]) < packet_numbers:
            binary, p = decode_hex(binary)
            packet["subpackets"].append(p)
        return binary, packet
    
def decode_hex(binary: str):
    version = int(binary[:3], 2)
    type_id = int(binary[3:6], 2)
    if type_id == 4:
        return read_literal_packet(binary)    
    return read_operator_packet(binary)

def get_versions(packets, versions):
    versions.append(packets["version"])
    for packet in packets.get("subpackets", []):
        versions = get_versions(packet, versions)
    return versions

def solve_part1(signal: str):
    binary = hex2bin(signal)
    remaining, decoded = decode_hex(binary)
    return sum(get_versions(decoded, []))

In [81]:
assert solve_part1("8A004A801A8002F478") == 16
assert solve_part1("620080001611562C8802118E34") == 12
assert solve_part1("C0015000016115A2E0802F182340") == 23
assert solve_part1("A0016C880162017C3686B18A3D4780") == 31

In [85]:
solutiona = solve_part1(data)
solutiona

908

In [86]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [97]:
def calculate_value(packet):
    if packet["type_id"] == 4:
        return packet["literal"]
    elif packet["type_id"] == 0:
        return sum([calculate_value(p) for p in packet.get("subpackets", [])])
    elif packet["type_id"] == 1:
        return np.product([calculate_value(p) for p in packet.get("subpackets", [])])
    elif packet["type_id"] == 2:
        return min([calculate_value(p) for p in packet.get("subpackets", [])])
    elif packet["type_id"] == 3:
        return max([calculate_value(p) for p in packet.get("subpackets", [])])
    elif packet["type_id"] == 5:
        return int(calculate_value(packet["subpackets"][0]) > calculate_value(packet["subpackets"][1]))
    elif packet["type_id"] == 6:
        return int(calculate_value(packet["subpackets"][0]) < calculate_value(packet["subpackets"][1]))
    elif packet["type_id"] == 7:
        return int(calculate_value(packet["subpackets"][0]) == calculate_value(packet["subpackets"][1]))
        

def solve_part2(signal: str):
    binary = hex2bin(signal)
    remaining, decoded = decode_hex(binary)
    return calculate_value(decoded)

In [98]:
assert solve_part2("C200B40A82") == 3
assert solve_part2("04005AC33890") == 54
assert solve_part2("880086C3E88112") == 7
assert solve_part2("CE00C43D881120") == 9
assert solve_part2("D8005AC2A8F0") == 1
assert solve_part2("F600BC2D8F") == 0
assert solve_part2("9C005AC2F8F0") == 0
assert solve_part2("9C0141080250320F1802104A08") == 1

In [99]:
solutionb = solve_part2(data)
solutionb

10626195124371

In [100]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 16! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 17

In [3]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=17)
data = puzzle.input_data
import re

In [2]:
data

'target area: x=287..309, y=-76..-48'

In [140]:
def read_data(data: str) -> tuple:
    match = re.match(r'target area: x=(?P<xmin>\d+)..(?P<xmax>\d+), y=(?P<ymax>-?\d+)..(?P<ymin>-?\d+)', data)
    return {k: int(v) for k, v in match.groupdict().items()}

def step(position: tuple, velocity: tuple) -> tuple:
    x, y = position
    vx, vy = velocity
    x += vx
    y += vy
    vy -= 1
    if vx > 0:
        vx -= 1
    elif vx < 0:
        vx += 1
    return (x, y), (vx, vy)

def walk(target, position, velocity, ymax):
    position, velocity = step(position, velocity)
    ymax = max(ymax, position[1])

    if position[0] > target["xmax"] and position[1] > target["ymin"]:
        return ymax, "over"
    elif position[0] < target["xmin"] and position[1] < target["ymax"]:
        return ymax, "under"
    elif position[0] > target["xmax"] or position[1] < target["ymax"]:
        return ymax, "other"
    if (
        position[0] >= target["xmin"] and 
        position[0] <= target["xmax"] and
        position[1] <= target["ymin"] and
        position[1] >= target["ymax"]
    ):
        return ymax, "hit"   
    return walk(target, position, velocity, ymax)

def solve(data):
    target = read_data(data)
    position = (0, 0)
    winner = 0
    hits = {}
    for vx in range(100):
        for vy in range(100):
            ymax, hit = walk(target, position, (vx, vy), 0)
            if hit == "hit":
                if ymax > winner:
                    winner_velocity = (vx, vy)
                winner = max(winner, ymax)
                
    print(winner_velocity)
    return winner


def solve2(data):
    target = read_data(data)
    position = (0, 0)
    winner = 0
    hits = 0
    for vx in range(1000):
        for vy in range(-1000, 1000):
            ymax, hit = walk(target, position, (vx, vy), 0)
            if hit == "hit":
                hits += 1
    return hits


In [134]:
assert solve('target area: x=20..30, y=-10..-5') == 45

(6, 9)


In [132]:
solutiona = solve(data)
solutiona

(24, 75)


2850

In [133]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [141]:
assert solve2('target area: x=20..30, y=-10..-5') == 112

In [142]:
solutionb = solve2(data)
solutionb

1117

In [143]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 17! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 18

In [522]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=18)
data = puzzle.input_data.split()
import math
from itertools import combinations

In [487]:
def read_number(number: str):
    """Read a Snailfish number (provided as sting) and return a map containing 
    all digits and their depth in the nested structure"""
    depth = 0
    di = []  # digits
    dm = []  # nested depth map
    ix = []  # indeces to get to this number
    d = ''
    for i, char in enumerate(number):
        if char == "[":
            depth += 1
        elif char == "]":
            if d:
                di.append(int(d))
                dm.append(depth)
                ix.append(i-len(d))
                d = ''
            depth -= 1

        elif char.isnumeric():
            d += char
        elif char == "," and d:
            di.append(int(d))
            dm.append(depth)
            ix.append(i-len(d))
            d = ''
            
    return di, dm, ix

In [494]:
def replace(number, position, replacement):
    if isinstance(position, int):
        position = (position, position+1)
    return number[:position[0]] + replacement + number[position[1]:]

def explode(number, debug = False):
    di, dm, ix = read_number(number)
    if debug:
        print(di)
        print(dm)
    prios = []

    needs_exploding = [d > 4 for d in dm]
    index = needs_exploding.index(True)
    if debug:
        print(f"Exploding index {index}")
    assert dm[index] == dm[index + 1]

    if index > 0:
        replacement = str(di[index-1] + di[index])
        if len(replacement) > 1:
            prios.append("split")
        number = replace(number, (ix[index-1], ix[index-1]+len(str(di[index-1]))), replacement)
        di, dm, ix = read_number(number)

    if index < (len(di) - 2):
        replacement = str(di[index+2] + di[index+1])
        if debug:
            print(f"Replacing right with {replacement}")
        if len(replacement) > 1:
            prios.append("split")
        number = replace(number, (ix[index+2], ix[index+2]+len(str(di[index+2]))), replacement)
        di, dm, ix = read_number(number)

    i = ix[index]
    number = replace(number, (i-1, i+2+len(str(di[index]))+len(str(di[index+1]))), '0')
    if debug:
        print(f"Exploded to: {number}")
    return number, prios
    
def split(number, debug=False):
    di, dm, ix = read_number(number)
    needs_splitting = [d > 9 for d in di]
    prios = []
    
    index = needs_splitting.index(True)
    number = replace(number, (ix[index], ix[index]+len(str(di[index]))), f"[{math.floor(di[index]/2)},{math.ceil(di[index]/2)}]")
    if dm[index] >= 4:
        prios.append("explode")
    if debug:
        print(f"Split to: {number}")
    return number, prios


def reduce(number, prios: list, debug = False):
    di, dm, ix = read_number(number)

    if 5 in dm:
        return reduce(*explode(number, debug), debug)

    else:
        needs_splitting = [d > 9 for d in di]
        if any(needs_splitting):
            return reduce(*split(number, debug), debug)

    return number


In [512]:
def pair_magnitude(pair):
    return 3*pair[0] + 2*pair[1]

def magnitude(e):
    if isinstance(e[0], list):
        e[0] = magnitude(e[0])
    if isinstance(e[1], list):
        e[1] = magnitude(e[1])
    return pair_magnitude(e)
    

def solve(data):
    number = data[0]
    for line in data[1:]:
        number = reduce(f"[{number},{line}]", [])
    return magnitude(eval(number))

In [515]:
example = [l.strip() for l in open('day18.example.input').readlines()]

In [516]:
assert solve(example) == 4140

In [520]:
solutiona = solve(data)
solutiona

4116

In [521]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [527]:
def solveb(data):
    largest = 0
    for combination in combinations(data, 2):
        mag = magnitude(eval(reduce(f"[{combination[0]},{combination[1]}]", [])))
        largest = max(largest, mag)
    for combination in combinations(data[::-1], 2):
        mag = magnitude(eval(reduce(f"[{combination[0]},{combination[1]}]", [])))
        largest = max(largest, mag)
    return largest

In [528]:
assert solveb(example) == 3993

In [529]:
solutionb = solveb(data)
solutionb

4638

In [530]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 18! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 19

In [821]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=19)
data = puzzle.input_data.split("\n")
import re
from collections import defaultdict
from itertools import combinations
import numpy as np

In [562]:
example = [l.strip() for l in open('day19.example.input').readlines()]

In [587]:
def read(data: list):
    scanners = defaultdict(list)
    for row in data:
        if row.startswith("---"):
            scanner_id = int(re.match(r"^---\sscanner\s(\d+)\s---$", row).groups()[0])
        elif row:
            scanners[scanner_id].append(np.array(eval(row)))
    return scanners

In [833]:
def dist(p):
    return sum([p[0]**2, p[1]**2, p[2]**2])

def calculate_distances(points):
    dists = []
    for point in points:
        distances = [dist(a-point) for a in points]
        dists.append(distances)
    return dists

def match(scannera, scannerb):
    distsa = calculate_distances(scannera)
    distsb = calculate_distances(scannerb)
    for pa, da in zip(scannera, distsa):
        for pb, db in zip(scannerb, distsb):
            matches = [d in da for d in db]
            if sum(matches) > 11:
                return pa, da, pb, db

def find_transform(scannera, scannerb):
    pa, da, pb, db = match(scannera, scannerb)
    for i, d in enumerate(db):
        if d in da and d != 0:
            ai = da.index(d)
            pa2 = scannera[ai]
            pb2 = scannerb[i]
            break
    diffa = pa - pa2
    diffb = pb - pb2
    indices = []
    for i in diffa:
        indices.append(int(np.where(np.abs(diffb) == abs(i))[0]))
    signs = diffa/(diffb[indices])
    summations = pa - pb[indices] * signs
    transformedb = pb[indices] * signs + summations
    return indices, signs, summations

def transform(scanner, transformations):
    transformed = []
    for point in scanner:
        for indices, signs, summations in transformations[::-1]:
            point = point[indices] * signs + summations
        transformed.append(point.astype(int))
    return transformed

def solve(data):
    scanners = read(data)
    points = set([tuple(i) for i in scanners[0]])
    transforms = {0: []}
    while len(transforms) < len(scanners):
        for a, b in combinations(scanners.keys(), 2):
            if a in transforms and b not in transforms and match(scanners[a], scanners[b]):
                transforms[b] = transforms[a] + [find_transform(scanners[a], scanners[b])]
            if b in transforms and a not in transforms and match(scanners[b], scanners[a]):
                transforms[a] = transforms[b] + [find_transform(scanners[b], scanners[a])]
    for i, transformations in transforms.items():
        transformed = transform(scanners[i], transformations)
        points.update(set([tuple(i) for i in transformed]))
    return len(points)

In [834]:
assert solve(example) == 79

In [835]:
solutiona = solve(data)
solutiona

512

In [832]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [863]:
def solveb(data):
    scanners = read(data)
    base = np.array([0, 0, 0])
    scanner_positions = []
    transforms = {0: []}
    while len(transforms) < len(scanners):
        for a, b in combinations(scanners.keys(), 2):
            if a in transforms and b not in transforms and match(scanners[a], scanners[b]):
                transforms[b] = transforms[a] + [find_transform(scanners[a], scanners[b])]
            if b in transforms and a not in transforms and match(scanners[b], scanners[a]):
                transforms[a] = transforms[b] + [find_transform(scanners[b], scanners[a])]
    
    for i, transformations in transforms.items():
        scanner_positions.append(transform([base], transformations)[0])
    
    max_dist = 0
    for c in combinations(scanner_positions, 2):
        max_dist = max(max_dist, sum(abs(c[0]-c[1])))
    return max_dist

In [864]:
assert solveb(example) == 3621

In [865]:
solutionb = solveb(data)
solutionb

16802

In [866]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 19! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 20

In [943]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=20)
data = puzzle.input_data.split("\n")
import scipy.ndimage as ndimage
import functools

In [938]:
example = [l.strip() for l in open('day20.example.input').readlines()]

In [969]:
def read(data, extend=100):
    lines = []
    for l in data[2:]:
        if l:
            lines.append([int(i) for i in l.replace(".", "0").replace("#", "1")])
    mp = np.array(lines)
    x, y = mp.shape
    mask = np.zeros((x+extend*2, y+extend*2))
    mask[extend:x+extend, extend:y+extend] = mp
    return data[0], mask

In [963]:
def solve(data, runs):
    footprint = np.array([[1,1,1],
                          [1,1,1],
                          [1,1,1]])

    def filter_func(values):
        binary = "".join([str(int(i)) for i in values])
        return algo[int(binary, 2)] == "#"

    algo, mp = read(data)
    for i in range(runs):
        mp = ndimage.generic_filter(mp, filter_func, footprint=footprint, mode='mirror')
    return int(sum(sum(mp)))

In [952]:
assert solve(example, 2) == 35

In [966]:
solutiona = solve(data, 2)
solutiona

5884

In [967]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [970]:
assert solve(example, 50) == 3351

In [971]:
solutionb = solve(data, 50)
solutionb

19043

In [972]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 20! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 21

In [80]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=21)
puzzle.input_data.split("\n")
import itertools
import functools
from collections import defaultdict

In [1031]:
def solve(pos1, pos2):
    p1 = 0
    p2 = 0
    rolls = 0
    d = 0
    while not any([p1 > 999, p2 > 999]):
        s = 0
        for i in range(d, d+3):
            s += i%100 + 1
        rolls += 3
        pos1 = ((pos1+s-1) % 10) + 1
        p1 += pos1
        s = 0
        for i in range(d+3, d+6):
            s += i%100 + 1
        rolls += 3
        pos2 = ((pos2+s-1) % 10) + 1
        p2 += pos2
        d = (d+6)%100
    if p1 > p2:
        print(f"Player 1 wins with a score of {p1} after rolling {rolls - 3} times")
        return (p2 - pos2) * (rolls - 3)
    print(f"Player 2 wins with a score of {p2} after rolling {rolls} times")
    return p1 * rolls

In [1032]:
assert solve(4, 8) == 739785

Player 1 wins with a score of 1000 after rolling 993 times


In [1033]:
solutiona = solve(4, 9)
solutiona

Player 1 wins with a score of 1000 after rolling 993 times


903630

In [1034]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [68]:
# Magic from https://github.com/Farbfetzen/Advent_of_Code/blob/main/python/2021/day21.py
# this was in the same direction as I was trying until I hit max recursion errors too often...
# too much magic for myself to accept using it... :) 

@functools.cache
def roll(pos1, p1, pos2, p2):
    if p2 > 20:
        return 1, 0
    win1 = 0
    win2 = 0
    for rolls in itertools.product((1, 2, 3), repeat=3):
        npos1 = (pos1 + sum(rolls) - 1 ) % 10 + 1
        np1 = p1 + npos1
        wins = roll(pos2, p2, npos1, np1)
        win1 += wins[1]
        win2 += wins[0]
    return win1, win2

In [86]:
dice = {3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}

def play(game, unis, games, wins):
    (pos1, p1), (pos2, p2) = game
    for d1, u1 in dice.items():
        npos1 = (pos1 + d1 - 1) % 10 + 1
        np1 = p1 + npos1
        if np1 > 20:
            wins[0] += u1 * unis
        else:
            for d2, u2 in dice.items():
                npos2 = (pos2 + d2 - 1 ) % 10 + 1
                np2 = p2 + npos2
                if np2 > 20:
                    wins[1] += u2 * u1 * unis
                else:
                    games[((npos1, np1), (npos2, np2))] += u1 * u2 * unis
    return games, wins
                

def solveb(pos1, pos2):
    games = {((pos1, 0), (pos2, 0)): 1}
    wins = [0, 0]
    while games:
        new_games = defaultdict(int)
        for game, unis in games.items():
            new_games, wins = play(game, unis, new_games, wins)
        games = new_games
    return max(wins)

In [87]:
assert solveb(4, 8) == 444356092776315

In [88]:
solutionb = solveb(4, 9)
solutionb

303121579983974

In [89]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 21! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 22

In [4]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=22)
data = puzzle.input_data.split("\n")
import numpy as np
import re
import itertools

In [5]:
example1 = [l.strip() for l in open('day22.example1.input').readlines()]
example2 = [l.strip() for l in open('day22.example2.input').readlines()]

In [6]:
def read(data):
    steps = []
    m = re.compile(r'(?P<switch>on|off) x=(?P<xmin>-?\d+)..(?P<xmax>-?\d+),y=(?P<ymin>-?\d+)..(?P<ymax>-?\d+),z=(?P<zmin>-?\d+)..(?P<zmax>-?\d+)')
    for line in data:
        step = m.match(line).groupdict()
        for k, v in step.items():
            if "m" in k:
                step[k] = int(v)
        steps.append(step)
    return steps

In [47]:
def filter_steps(steps):
    for s in steps.copy():
        if any(
            [i < -50 for i in [s["xmin"], s["ymin"], s["zmin"]]]\
            + [a > 50 for a in [s["xmax"], s["ymax"], s["zmax"]]]
        ):
            steps.remove(s)
    return steps

def solve(data):
    steps = read(data)
    steps = filter_steps(steps)
    base = np.zeros((101, 101, 101))
    for s in steps:
        switch = 0 if s["switch"] == "off" else 1
        base[s["xmin"]+50:s["xmax"]+51,s["ymin"]+50:s["ymax"]+51,s["zmin"]+50:s["zmax"]+51] = switch
    return int(sum(sum(sum(base))))

In [48]:
assert solve(example1) == 39
assert solve(example2) == 590784

In [51]:
solutiona = solve(data)
solutiona

543306

In [52]:
puzzle.answer_a = solutiona

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [127]:
class Cube:

    @classmethod
    def from_step(cls, step: dict):
        c = cls()
        c.x1 = step["xmin"]
        c.x2 = step["xmax"]
        c.y1 = step["ymin"]
        c.y2 = step["ymax"]
        c.z1 = step["zmin"]
        c.z2 = step["zmax"]
        c.switch = 1 if step["switch"] == "on" else 0
        return c
    
    @staticmethod
    def is_cube(t):
        return t[0] <= t[1] and t[2] <= t[3] and t[4] <= t[5]
    
    @classmethod
    def cubelet(cls, t):
        c = cls()
        c.x1 = t[0]
        c.x2 = t[1]
        c.y1 = t[2]
        c.y2 = t[3]
        c.z1 = t[4]
        c.z2 = t[5]
        c.switch = 1
        return c
        
    def overlaps(self, cube):
        x_overlap = any((self.x1 > cube.x2, self.x2 < cube.x1)) == False
        y_overlap = any((self.y1 > cube.y2, self.y2 < cube.y1)) == False
        z_overlap = any((self.z1 > cube.z2, self.z2 < cube.z1)) == False
        return all([x_overlap, y_overlap, z_overlap])
    
    def recube(self, cube):
        # split self in subcubes after removing the overlap with the provided cube
        cubelets = []
        dims = [
            (self.x1, cube.x1-1, self.y1, self.y2, self.z1, self.z2),
            (max(cube.x1, self.x1), self.x2, self.y1, cube.y1-1, self.z1, self.z2),
            (max(cube.x1, self.x1), self.x2, max(cube.y1, self.y1), self.y2, self.z1, cube.z1-1),
            (cube.x2+1, self.x2, max(self.y1, cube.y1), self.y2, max(self.z1, cube.z1), self.z2),
            (max(cube.x1, self.x1), min(self.x2, cube.x2), cube.y2+1, self.y2, max(cube.z1, self.z1), self.z2),
            (max(cube.x1, self.x1), min(self.x2, cube.x2), max(self.y1, cube.y1), min(cube.y2, self.y2), cube.z2+1, self.z2)
        ]
        for i, dim in enumerate(dims):
            if Cube.is_cube(dim):
                cubelets.append(Cube.cubelet(dim))
        return cubelets
    
    def count(self):
        x = self.x2 - self.x1 + 1
        y = self.y2 - self.y1 + 1
        z = self.z2 - self.z1 + 1
        return x*y*z
    
    def __str__(self):
        return f"({self.x1},{self.x2}) ({self.y1},{self.y2}) ({self.z1},{self.z2})"
    
def solveb(data):
    steps = read(data)
    cubes = [Cube.from_step(step) for step in steps]
    processed = []
    for i, cube in enumerate(cubes):
        new = []
        for p in processed:
            if p.overlaps(cube):
                cubelets = p.recube(cube)
                new.extend(cubelets)
            else:
                new.append(p)
        if cube.switch == 1:
            new.append(cube)
        processed = new

    return sum([p.count() for p in processed if p.switch == 1])

In [139]:
assert solveb(example1) == 39
assert solveb(example2[:-2]) == 590784

In [135]:
example3 = [l.strip() for l in open('day22.example3.input').readlines()]

In [141]:
assert solveb(example3) == 2758514936282235

In [142]:
solutionb = solveb(data)
solutionb

1285501151402480

In [143]:
puzzle.answer_b = solutionb

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 22! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 23

In [313]:
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=23)
data = puzzle.input_data.split("\n")
import itertools
from copy import deepcopy

In [146]:
data

['#############',
 '#...........#',
 '###A#D#C#A###',
 '  #C#D#B#B#',
 '  #########']

In [464]:
class Amph:
    name: str
    cost: int
    homey: int
    position: tuple
    energy: int = 0
    moves: int = 0
    
    def __init__(self, position):
        self.position = position
    
    @property
    def is_home(self):
        return self.position[1] == self.homey
    
    def move(self, position):
        self.energy += (abs(position[0]-self.position[0])+abs(position[1]-self.position[1])) * self.cost
        self.position = position        
        self.moves += 1
    
    def available_positions(self, game) -> list:
        if self.moves == 2:
            return []
        
        if self.moves == 1: # has moved, can only return to its own room
            if game.room_available(self.homey):
                if not game.occupied((2, self.homey)):
                    return [(2, self.homey)]
                else:
                    return [(1, self.homey)]
            else:
                return []
        
        if self.position[0] == 2 and game.occupied((1, self.position[1])): # locked by amphipod above
            return []

        positions = []
        y = self.homey-1
        while (0, y) in game.positions and not game.occupied((0, y)):
            if (0, y) not in game.transfer_positions:
                positions.append((0, y))
            y -= 1
        y = self.homey + 1
        while (0, y) in game.positions and not game.occupied((0, y)):
            if (0, y) not in game.transfer_positions:
                positions.append((0, y))
            y += 1
        return positions
    
    def at_home(self) -> bool:
        return self.position[1] == self.homey

    def __str__(self):
        return f"{self.name} @ ({self.position[0]},{self.position[1]}) - energy: {self.energy}"
    
class Amber(Amph):
    name = "Amber"
    short = "A"
    cost = 1
    homey = 2
    
class Bronze(Amph):
    name = "Bronze"
    short = "B"
    cost = 10
    homey = 4
    
class Copper(Amph):
    name = "Copper"
    short = "C"
    cost = 100
    homey = 6

class Desert(Amph):
    name = "Desert"
    short = "D"
    cost = 1000
    homey = 8

class Game:
    positions= [
        (0, 0),
        (0, 1),
        (0, 2),
        (0, 3),
        (0, 4),
        (0, 5),
        (0, 6),
        (0, 7),
        (0, 8),
        (0, 9),
        (0, 10),
        (1, 2),
        (2, 2),
        (1, 4),
        (2, 4),
        (1, 6),
        (2, 6),
        (1, 8),
        (2, 8)
    ]
    transfer_positions = [
        (0, 2),
        (0, 4), 
        (0, 6),
        (0, 8)
    ]
        
    def __init__(self, data):
        self.amphipods = []
        mp = []
        for d in data:
            for c in ["A", "B", "C", "D"]:
                d = d.replace(c, ".")
            mp.append(d)
        self.empty = [list(d) for d in mp]
    
    @property
    def mapping(self):
        e = self.empty.copy()
        for amph in self.amphipods:
            e[amph.position[0]+1][amph.position[1]+1] = amph.short
        return e
    
    @property
    def data(self):
        return ["".join(e) for e in self.mapping]
    
    @classmethod
    def from_data(cls, data):
        game = cls(data)
        for row in [1, 2, 3]:
            for i, c in enumerate(data[row]):
                if c == "A":
                    game.add_amphipod(Amber((row-1, i-1)))
                if c == "B":
                    game.add_amphipod(Bronze((row-1, i-1)))
                if c == "C":
                    game.add_amphipod(Copper((row-1, i-1)))
                if c == "D":
                    game.add_amphipod(Desert((row-1, i-1)))
        return game        
    
    def add_amphipod(self, amphipod):
        self.amphipods.append(amphipod)
    
    def occupied(self, position) -> bool:
        return any([position == a.position for a in self.amphipods])
    
    def get_amphipod(self, position):
        for amph in self.amphipods:
            if amph.position == position:
                return amph
    
    def room_available(self, homey) -> False:
        for amph in self.amphipods:
            if amph.position[1] == homey and not amph.at_home():
                return False  # another amphipod is still in the room
        return self.occupied((1, homey)) == False
    
    @property
    def finished(self):
        return all([amph.is_home for amph in self.amphipods])

    def energy(self):
        return sum([amph.energy for amph in self.amphipods])
    
    def __str__(self):
        return "\n".join(["".join(d) for d in self.mapping])

In [474]:
def solve(data):
    games = [Game.from_data(data)]
    min_energy = 0
    finished_games = []
    while games:
        new_games = []
        for game in games:
            if finished_games and game.energy() > min_energy:
                continue
            moves = []
            for amph in game.amphipods:
                for position in amph.available_positions(game):
                    moves.append((amph.position, position))
#             print(f"Detected {len(moves)} possible moves")
            for current_position, new_position in moves:
                g = Game.from_data(game.data)
                moved = False
                for amph in g.amphipods:
                    if amph.position == current_position:
                        amph.move(new_position)
                        moved = True
                if not moved:
                    print("ohoh")
                if g.finished:
                    print("Finished a game!")
                    finished_games.append(g)
                else:
                    new_games.append(g)
#             print(f"There are {len(new_games)} new games and {len(finished_games)} finished games")

        games = new_games
        print(f"Playing {len(games)} games")
        if len(games)> 10000:
            return games
        if finished_games:
            min_energy = min([game.energy() for game in finished_games])
    return min([game.energy() for game in finished_games])
    

In [475]:
example = [l.strip("\n") for l in open('day23.example.input').readlines()]

In [476]:
a = solve(example)

Playing 28 games
Playing 516 games
Playing 7438 games
Playing 84191 games


In [478]:
g = Game.from_data(a[0].data)

In [479]:
print(g)

#############
#.A.B.C.D...#
###.#.#B#.###
  #.#D#C#A#
  #########


In [486]:
for a in g.amphipods:
    print(a)
    print(a.available_positions(g))

Amber @ (0,1) - energy: 0
[]
Bronze @ (0,3) - energy: 0
[]
Copper @ (0,5) - energy: 0
[]
Desert @ (0,7) - energy: 0
[(0, 9), (0, 10)]
Bronze @ (1,6) - energy: 0
[]
Desert @ (2,4) - energy: 0
[(0, 9), (0, 10)]
Copper @ (2,6) - energy: 0
[]
Amber @ (2,8) - energy: 0
[]


In [481]:
for a in g.amphipods:
    print(a)

Amber @ (0,1) - energy: 0
Bronze @ (0,3) - energy: 0
Copper @ (0,5) - energy: 0
Desert @ (0,7) - energy: 0
Bronze @ (1,6) - energy: 0
Desert @ (2,4) - energy: 0
Copper @ (2,6) - energy: 0
Amber @ (2,8) - energy: 0


In [477]:
for g in a[:10]:
    print(g)

#############
#.A.B.C.D...#
###.#.#B#.###
  #.#D#C#A#
  #########
#############
#A..B.C.D...#
###.#.#B#.###
  #.#D#C#A#
  #########
#############
#...B.C.D.D.#
###.#.#B#.###
  #A#.#C#A#
  #########
#############
#...B.C.D..D#
###.#.#B#.###
  #A#.#C#A#
  #########
#############
#.A.B.C.D...#
###.#.#B#.###
  #A#D#C#.#
  #########
#############
#A..B.C.D...#
###.#.#B#.###
  #A#D#C#.#
  #########
#############
#...B...C.D.#
###.#.#B#.###
  #A#D#C#A#
  #########
#############
#.A.B.C...D.#
###.#.#B#.###
  #.#D#C#A#
  #########
#############
#A..B.C...D.#
###.#.#B#.###
  #.#D#C#A#
  #########
#############
#...B.C.D.D.#
###.#.#B#.###
  #A#.#C#A#
  #########


In [442]:
def solve(data):
    game = read(data)
    games = []
    moves = []
    for amph in game.amphipods:
        for position in amph.available_positions(game):
            moves.append((amph.position, position))
    print(f"Found {len(moves)} available moves")
    for move in moves:
        g = Game.from_data(game.data)
        for amph in g.amphipods:
            if amph.position == move[0]:
                amph.move(move[1])
        games.append(g)
    return games
    